In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# Increase figure size
plt.rcParams['figure.figsize'] = [25, 10]

In [2]:
# Get CPI data for salary comparisons
CPI = pd.read_csv("CPI.csv").set_index("Year")
current_benchmark = CPI.loc[2016, "Ave."]
multiplier = current_benchmark / CPI["Ave."]
multiplier

Year
2019          NaN
2018     0.955800
2017     0.979145
2016     1.000000
2015     1.012619
          ...    
1917    18.750625
1916    22.019083
1915    23.763168
1914    24.000800
1913    24.243232
Name: Ave., Length: 107, dtype: float64

In [3]:
# Round salaries to nearest dollar
pd.options.display.float_format = '${:,.0f}'.format

In [4]:
con = sqlite3.connect("lahmansbaseballdb.sqlite")
cur = con.cursor()

In [5]:
# List of all tables in DB
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
cur.fetchall()

[('allstarfull',),
 ('appearances',),
 ('awardsmanagers',),
 ('awardsplayers',),
 ('awardssharemanagers',),
 ('awardsshareplayers',),
 ('batting',),
 ('battingpost',),
 ('collegeplaying',),
 ('divisions',),
 ('fielding',),
 ('fieldingof',),
 ('fieldingofsplit',),
 ('fieldingpost',),
 ('halloffame',),
 ('homegames',),
 ('leagues',),
 ('managers',),
 ('managershalf',),
 ('parks',),
 ('people',),
 ('pitching',),
 ('pitchingpost',),
 ('salaries',),
 ('schools',),
 ('seriespost',),
 ('teams',),
 ('teamsfranchises',),
 ('teamshalf',)]

In [6]:
# Read salary table into dataframe of same name
salaries = pd.read_sql_query("SELECT people.playerID, nameFirst, nameLast, yearID, teamID, salary FROM salaries INNER JOIN people ON salaries.playerID=people.playerID ORDER BY yearID, teamID, people.playerID", con)
salaries

,playerID,nameFirst,nameLast,yearID,teamID,salary
0,barkele01,Len,Barker,1985,ATL,"$870,000"
1,bedrost01,Steve,Bedrosian,1985,ATL,"$550,000"
2,benedbr01,Bruce,Benedict,1985,ATL,"$545,000"
3,campri01,Rick,Camp,1985,ATL,"$633,333"
4,ceronri01,Rick,Cerone,1985,ATL,"$625,000"
...,...,...,...,...,...,...
26423,strasst01,Stephen,Strasburg,2016,WAS,"$10,400,000"
26424,taylomi02,Michael,Taylor,2016,WAS,"$524,000"
26425,treinbl01,Blake,Treinen,2016,WAS,"$524,900"
26426,werthja01,Jayson,Werth,2016,WAS,"$21,733,615"


In [7]:
# Determine primary position by which position they played the most innings at
fielding = pd.read_sql_query("SELECT playerID, yearID, POS FROM fielding WHERE yearID in (SELECT yearID FROM salaries) GROUP BY yearID, playerID HAVING max(InnOuts) ORDER BY yearID, playerID", con)
fielding

,playerID,yearID,POS
0,aasedo01,1985,P
1,abregjo01,1985,P
2,ackerji01,1985,P
3,adamsri02,1985,SS
4,agostju01,1985,P
...,...,...,...
37077,zimmejo02,2016,P
37078,zimmery01,2016,1B
37079,zobribe01,2016,2B
37080,zuninmi01,2016,C


In [8]:
salary_with_pos = salaries.merge(fielding, on=["playerID", "yearID"])
salary_with_pos

,playerID,nameFirst,nameLast,yearID,teamID,salary,POS
0,barkele01,Len,Barker,1985,ATL,"$870,000",P
1,bedrost01,Steve,Bedrosian,1985,ATL,"$550,000",P
2,benedbr01,Bruce,Benedict,1985,ATL,"$545,000",C
3,campri01,Rick,Camp,1985,ATL,"$633,333",P
4,ceronri01,Rick,Cerone,1985,ATL,"$625,000",C
...,...,...,...,...,...,...,...
25468,strasst01,Stephen,Strasburg,2016,WAS,"$10,400,000",P
25469,taylomi02,Michael,Taylor,2016,WAS,"$524,000",OF
25470,treinbl01,Blake,Treinen,2016,WAS,"$524,900",P
25471,werthja01,Jayson,Werth,2016,WAS,"$21,733,615",OF


In [9]:
# Average salary by position over the years
salary_by_pos = pd.pivot_table(salary_with_pos, values="salary", index="yearID", columns="POS", margins=True)[:-1]
salary_by_pos_sum = pd.pivot_table(salary_with_pos, values="salary", index="yearID", columns="POS", aggfunc='sum', margins=True)[:-1]
salary_by_pos["All_Sum"] = salary_by_pos["All"] * 9
salary_by_pos

POS,1B,2B,3B,C,OF,P,SS,All,All_Sum
yearID,,,,,,,,,
1985,"$600,300","$453,178","$488,649","$460,943","$493,873","$452,657","$412,160","$475,503","$4,279,527"
1986,"$524,218","$376,198","$488,932","$335,972","$468,257","$387,571","$375,192","$416,092","$3,744,825"
1987,"$621,193","$404,226","$419,637","$397,397","$457,238","$409,389","$346,563","$429,616","$3,866,546"
1988,"$585,206","$412,314","$504,628","$360,827","$500,752","$438,188","$425,006","$457,691","$4,119,220"
1989,"$689,788","$413,057","$469,044","$393,663","$561,528","$490,982","$475,233","$503,798","$4,534,184"
1990,"$839,982","$582,485","$425,189","$386,396","$607,866","$496,478","$508,783","$533,413","$4,800,714"
1991,"$1,303,941","$726,085","$695,455","$543,000","$1,041,560","$896,680","$836,908","$892,959","$8,036,628"
1992,"$1,671,022","$991,518","$842,165","$704,514","$1,250,716","$1,061,762","$730,531","$1,063,749","$9,573,745"
1993,"$1,299,748","$869,743","$1,022,472","$645,398","$1,107,223","$961,962","$937,493","$985,732","$8,871,589"


In [10]:
salary_by_pos_sum

POS,1B,2B,3B,C,OF,P,SS,All
yearID,,,,,,,,
1985,"$27,013,500","$14,954,881","$17,102,731","$18,437,705","$61,734,064","$99,584,594","$13,189,114","$252,016,589"
1986,"$28,307,784","$15,424,108","$23,468,721","$22,174,149","$72,111,652","$118,596,687","$15,758,084","$295,841,185"
1987,"$25,468,924","$13,743,667","$18,883,666","$21,062,043","$64,470,500","$104,394,147","$14,902,188","$262,925,135"
1988,"$27,504,701","$17,317,189","$22,708,242","$22,732,107","$73,109,799","$116,996,162","$15,300,224","$295,668,424"
1989,"$32,420,038","$18,174,500","$21,107,000","$24,800,740","$85,913,712","$145,330,654","$21,385,500","$349,132,144"
1990,"$46,199,036","$28,541,750","$23,810,583","$29,752,500","$101,513,653","$177,242,505","$23,403,999","$430,464,026"
1991,"$58,677,332","$31,221,667","$30,600,000","$34,209,000","$143,735,332","$264,520,670","$32,639,417","$595,603,418"
1992,"$76,867,001","$46,601,332","$36,213,083","$50,020,501","$192,610,252","$340,825,485","$38,718,169","$781,855,823"
1993,"$71,486,167","$49,575,333","$54,191,000","$46,468,668","$221,444,564","$388,632,835","$54,374,567","$886,173,134"


In [11]:
teams = pd.read_sql_query("SELECT yearID, teamID, W, L, DivWin, WCWin, LgWin, WSWin FROM teams WHERE yearID>=1985", con)

In [12]:
salary_by_pos_team = pd.pivot_table(salary_with_pos, values="salary", index=["yearID","teamID"], columns="POS", aggfunc='sum', margins=True)[:-1]
salary_by_pos_team_results = salary_by_pos_team.merge(teams, on=["yearID", "teamID"])
salary_by_pos_team_results

,yearID,teamID,1B,2B,3B,C,OF,P,SS,All,W,L,DivWin,WCWin,LgWin,WSWin
0,1985,ATL,"$2,420,000","$455,000","$616,667","$1,170,000","$3,450,000","$5,945,333","$750,000","$14,807,000",66,96,N,None,N,N
1,1985,BAL,"$1,472,819","$480,000","$611,833","$854,167","$3,059,333","$3,832,560","$800,000","$11,110,712",83,78,N,None,N,N
2,1985,BOS,"$747,500","$597,500","$1,125,000","$477,500","$3,679,227","$3,322,500","$465,000","$10,414,227",81,81,N,None,N,N
3,1985,CAL,"$1,005,000","$966,667","$1,100,000","$883,000","$3,484,227","$5,032,333","$120,000","$12,591,227",90,72,N,None,N,N
4,1985,CHA,"$195,000","$1,242,333","$165,000","$885,000","$2,348,333","$4,950,512","$60,000","$9,846,178",85,77,N,None,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,2016,SLN,"$1,650,000","$5,250,000","$20,500,000","$17,275,000","$27,300,000","$58,887,000","$507,500","$131,369,500",86,76,N,N,N,N
914,2016,TBA,"$8,950,000","$2,250,000","$12,600,018","$2,013,600","$6,038,400","$24,203,192","$1,042,100","$57,097,310",68,94,N,N,N,N
915,2016,TEX,"$29,700,000","$522,700","$18,000,000","$3,044,000","$30,677,420","$54,249,770","$15,844,833","$152,038,723",95,67,Y,N,N,N
916,2016,TOR,"$14,421,300","$2,081,400","$11,650,000","$15,800,000","$17,942,900","$56,806,100","$20,000,000","$138,701,700",89,73,N,Y,N,N


In [13]:
# Get salary by position as percentage of total payroll
pd.options.display.float_format = '{:,.2}'.format
salary_by_pos_pct = salary_by_pos.iloc[:,:7].apply(lambda x: x / salary_by_pos["All_Sum"])
salary_by_pos_sum_pct = salary_by_pos_sum.iloc[:,:7].apply(lambda x: x / salary_by_pos_sum["All"])
salary_by_pos_sum_pct

POS,1B,2B,3B,C,OF,P,SS
yearID,,,,,,,
1985,0.11,0.059,0.068,0.073,0.24,0.4,0.052
1986,0.096,0.052,0.079,0.075,0.24,0.4,0.053
1987,0.097,0.052,0.072,0.08,0.25,0.4,0.057
1988,0.093,0.059,0.077,0.077,0.25,0.4,0.052
1989,0.093,0.052,0.06,0.071,0.25,0.42,0.061
1990,0.11,0.066,0.055,0.069,0.24,0.41,0.054
1991,0.099,0.052,0.051,0.057,0.24,0.44,0.055
1992,0.098,0.06,0.046,0.064,0.25,0.44,0.05
1993,0.081,0.056,0.061,0.052,0.25,0.44,0.061


In [14]:
#salary_by_pos.index.name = "Year"
## you get ax from here
#ax = salary_by_pos.iloc[:, :7].plot()
#vals = ax.get_yticks()

## manipulate
#ax.set_ylabel("Salary")
#ax.legend(bbox_to_anchor=(1, 0.5))
#ax.set_yticklabels(['${:,.0f}'.format(x) for x in vals])
#ax.set_title("Average Player Salary by Position Over Time (1985-2016)")

In [15]:
## Base salaries off of 2016 value of dollar
#iter = 0
#for index in salary_by_pos.index:
#    salary_by_pos.iloc[iter,:] = salary_by_pos.iloc[iter,:] * multiplier[index]
#    iter += 1
    
## you get ax from here
#ax = salary_by_pos.iloc[:, :7].plot()
#vals = ax.get_yticks()

## manipulate
#ax.set_ylabel("Salary")
#ax.legend(bbox_to_anchor=(1, 0.5))
#ax.set_yticklabels(['${:,.0f}'.format(x) for x in vals])
#ax.set_title("Average Player Salary by Position Over Time (1985-2016)\n2016 Dollars")

In [16]:
#salary_by_pos_sum_pct.index.name = "Year"
## you get ax from here
#ax = salary_by_pos_sum_pct.plot()
#vals = ax.get_yticks()

## manipulate
#ax.set_ylabel("Percentage of Salary")
#ax.set_yticklabels(['{:,.0%}'.format(x) for x in vals])
#ax.legend(bbox_to_anchor=(1, 0.5))
#ax.set_title("Percentage of Salary by Position Over Time (1985-2016)")

In [17]:
positions = ["P", "C", "1B", "2B", "3B", "SS", "OF"]
WAR_by_position = pd.DataFrame()

for position in positions:
    if position == "P":
        # Interested in pitching WAR
        stat = "pit"
    else:
        # Interested in hitting/baserunning/fielding WAR
        stat = "bat"
    url = "https://www.fangraphs.com/leaders.aspx?pos=%s&stats=%s&lg=all&qual=0&type=8&season=2016&month=0&season1=1985&ind=0&team=0,ss&rost=0&age=0&filter=&players=0&startdate=&enddate=&page=1_50" % (position.lower(), stat)
    # Create dataframe (dropping first column and last row)
    df = pd.read_html(url)[16].iloc[:-1, 1:]
    # Rename columns to nicer format
    df.columns = df.columns.droplevel(0)
    # Add Position column before pivoting
    df["Position"] = position.upper()
    WAR_by_position = WAR_by_position.append(df, sort=False)
    
WAR_by_position = WAR_by_position.pivot(index="Season", columns="Position", values="WAR")
WAR_by_position

Position,1B,2B,3B,C,OF,P,SS
Season,,,,,,,
1985,64.3,62.7,83.6,62.8,198.7,371.8,42.0
1986,76.6,53.4,78.3,57.7,198.8,372.0,40.0
1987,87.5,54.0,80.2,37.7,221.2,372.5,36.6
1988,72.6,52.8,66.9,57.3,198.6,371.2,57.3
1989,74.3,69.7,68.0,51.2,193.3,372.1,49.2
1990,84.9,63.4,61.9,62.8,211.1,372.5,48.7
1991,70.1,64.0,82.7,48.0,203.0,372.3,50.4
1992,62.5,70.4,69.1,56.1,197.4,372.7,55.5
1993,81.4,66.9,73.1,66.5,211.7,401.3,62.9


TO DO: NEED WAR PER DOLLAR

In [18]:
con.close()